Note that need tensorflow 1.15.2

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import scipy.io
#from scipy.interpolate import griddata
import time
#from itertools import product, combinations

In [2]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [34]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, t, u1, u2,u3,u4,u5,u6, F, tau, layers):
        
        X = t
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.t = X[:,0:1]
        self.F = F[:,0:1]
        self.tau = tau[:,0:1]
        
        self.u1 = u1
        self.u2 = u2
        self.u3 = u3
        self.u4 = u4
        self.u5 = u5
        self.u6 = u6
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.bv = tf.Variable([4.0], dtype=tf.float32)    
        self.bo = tf.Variable([10.0], dtype=tf.float32)    
        self.m = tf.Variable([11.0], dtype=tf.float32)  
        self.I = tf.Variable([10.0], dtype=tf.float32)    
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.F_tf = tf.placeholder(tf.float32, shape=[None, self.F.shape[1]])
        self.tau_tf = tf.placeholder(tf.float32, shape=[None, self.tau.shape[1]])
        
        self.u1_tf = tf.placeholder(tf.float32, shape=[None, self.u1.shape[1]])
        self.u2_tf = tf.placeholder(tf.float32, shape=[None, self.u2.shape[1]])
        self.u3_tf = tf.placeholder(tf.float32, shape=[None, self.u3.shape[1]])
        self.u4_tf = tf.placeholder(tf.float32, shape=[None, self.u4.shape[1]])
        self.u5_tf = tf.placeholder(tf.float32, shape=[None, self.u5.shape[1]])
        self.u6_tf = tf.placeholder(tf.float32, shape=[None, self.u6.shape[1]])
        
        self.u1_pred,self.u2_pred,self.u3_pred,self.u4_pred,self.u5_pred,self.u6_pred,\
        self.f_u1_pred ,self.f_u2_pred,self.f_u3_pred,self.f_u4_pred,self.f_u5_pred,self.f_u6_pred = \
            self.net_NS(self.t_tf, self.F_tf, self.tau_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.u1_tf - self.u1_pred)) + \
                    tf.reduce_sum(tf.square(self.u2_tf - self.u2_pred)) + \
                    tf.reduce_sum(tf.square(self.u3_tf - self.u3_pred)) + \
                    tf.reduce_sum(tf.square(self.u4_tf - self.u4_pred)) + \
                    tf.reduce_sum(tf.square(self.u5_tf - self.u5_pred)) + \
                    tf.reduce_sum(tf.square(self.u6_tf - self.u6_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u1_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u2_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u3_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u4_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u5_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u6_pred))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer(learning_rate=0.01)
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = X
        #H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
        
    def net_NS(self, t, F, tau):
        bv = self.bv
        bo = self.bo
        m = self.m
        I = self.I
        g = 9.8
        
        state = self.neural_net(tf.concat([t], 1), self.weights, self.biases)
        u1 = state[:,0:1]
        u2 = state[:,1:2]
        u3 = state[:,2:3]
        u4 = state[:,3:4]
        u5 = state[:,4:5]
        u6 = state[:,5:6]
        
        
        u1_t = tf.gradients(u1, t)[0]
        u2_t = tf.gradients(u2, t)[0]
        u3_t = tf.gradients(u3, t)[0]
        u4_t = tf.gradients(u4, t)[0]
        u5_t = tf.gradients(u5, t)[0]
        u6_t = tf.gradients(u6, t)[0]

        f_u1 = u1_t - u1 
        f_u2 = u2_t - u2
        f_u3 = u3_t - u3
        f_u4 = u4_t + u4*bv/m + tf.math.multiply(tf.math.sin(u3)/m, tf.convert_to_tensor(F))
        f_u5 = u5_t + u5*bv/m + g - tf.math.multiply(tf.math.cos(u3)/m, tf.convert_to_tensor(F))
        f_u6 = u6_t + u6*bo/I - tf.convert_to_tensor(tau)/I
        
        return u1, u2, u3, u4, u5, u6, f_u1, f_u2, f_u3, f_u4, f_u5, f_u6
    
    def callback(self, loss, bv, bo, m, I):
        print('Loss: %.3e, bv: %.2f, bo: %.2f, m: %.2f, I: %.5f' % (loss, bv, bo, m, I))
      
    def train(self, nIter): 

        tf_dict = {self.t_tf: self.t, self.u1_tf: self.u1, self.u2_tf: self.u2, self.u3_tf: self.u3, 
                   self.u4_tf: self.u4, self.u5_tf: self.u5, self.u6_tf: self.u6, self.F_tf: self.F, self.tau_tf: self.tau}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                bv_value = self.sess.run(self.bv)
                bo_value = self.sess.run(self.bo)
                m_value = self.sess.run(self.m)
                I_value = self.sess.run(self.I)
                print('It: %d, Loss: %.3e, bv: %.2f, bo: %.2f, m: %.2f, I: %.5f, Time: %.2f' % 
                      (it, loss_value, bv_value, bo_value, m_value, I_value, elapsed))
                start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.bv, self.bo, self.m, self.I],
                                loss_callback = self.callback)
            
    
    #def predict(self, t_star):
        
    #    tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star}
        
    #    u_star = self.sess.run(self.u_pred, tf_dict)
    #    v_star = self.sess.run(self.v_pred, tf_dict)
    #    p_star = self.sess.run(self.p_pred, tf_dict)
        
    #    return u_star, v_star, p_star
              

In [35]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [36]:
# Load Data

# read data

filepath = '../Data/'
DATA_dict = dict()
num_traj = 40
for i in range(num_traj):
    num = i+41;
    if num >= 10:
        str_num = str(num) # no leading zero
    else:
        str_num = '0'+str(num) # add leading zero
    filename = 'trajdata_'+str_num+'.csv'
    with open(filepath+filename, 'r') as f:
        # remove top row, and store the rest in dictionary
        DATA_dict[i] = np.genfromtxt(f, dtype='f4', delimiter=',', skip_header=1)

In [37]:
idx = 23
t = DATA_dict[idx][:,0:1]
u1 = DATA_dict[idx][:,1:2]
u2 = DATA_dict[idx][:,2:3]
u3 = DATA_dict[idx][:,3:4]
u4 = DATA_dict[idx][:,4:5]
u5 = DATA_dict[idx][:,5:6]
u6 = DATA_dict[idx][:,6:7]
F = DATA_dict[idx][:,7:8]
tau = DATA_dict[idx][:,8:9]

In [38]:
layers = [1, 10, 10, 6]

# Training
model = PhysicsInformedNN(t, u1, u2, u3, u4, u5, u6, F, tau, layers)
model.train(200000)
    

Device mapping:

It: 0, Loss: 4.567e+05, bv: 4.01, bo: 9.99, m: 11.01, I: 10.01000, Time: 0.41
It: 10, Loss: 4.224e+05, bv: 4.02, bo: 9.92, m: 11.11, I: 10.10984, Time: 0.07
It: 20, Loss: 3.996e+05, bv: 4.02, bo: 9.88, m: 11.20, I: 10.20908, Time: 0.07
It: 30, Loss: 3.898e+05, bv: 3.98, bo: 9.94, m: 11.30, I: 10.30728, Time: 0.08
It: 40, Loss: 3.783e+05, bv: 3.95, bo: 10.04, m: 11.38, I: 10.40417, Time: 0.07
It: 50, Loss: 3.664e+05, bv: 3.94, bo: 10.18, m: 11.47, I: 10.49960, Time: 0.06
It: 60, Loss: 3.543e+05, bv: 4.00, bo: 10.31, m: 11.55, I: 10.59341, Time: 0.08
It: 70, Loss: 3.445e+05, bv: 4.13, bo: 10.41, m: 11.63, I: 10.68570, Time: 0.07
It: 80, Loss: 3.367e+05, bv: 4.29, bo: 10.38, m: 11.70, I: 10.77673, Time: 0.08
It: 90, Loss: 3.283e+05, bv: 4.44, bo: 10.30, m: 11.77, I: 10.86644, Time: 0.06
It: 100, Loss: 3.188e+05, bv: 4.57, bo: 10.29, m: 11.83, I: 10.95450, Time: 0.06
It: 110, Loss: 3.076e+05, bv: 4.70, bo: 10.39, m: 11.89, I: 11.04058, Time: 0.08
It: 120, Loss: 2.952e+05, 

It: 1020, Loss: 5.868e+04, bv: 5.18, bo: 13.01, m: 10.37, I: 13.62220, Time: 0.09
It: 1030, Loss: 5.864e+04, bv: 5.19, bo: 12.95, m: 10.37, I: 13.62520, Time: 0.06
It: 1040, Loss: 5.860e+04, bv: 5.19, bo: 12.89, m: 10.37, I: 13.62817, Time: 0.06
It: 1050, Loss: 5.857e+04, bv: 5.20, bo: 12.84, m: 10.36, I: 13.63111, Time: 0.08
It: 1060, Loss: 5.853e+04, bv: 5.20, bo: 12.78, m: 10.36, I: 13.63403, Time: 0.07
It: 1070, Loss: 5.850e+04, bv: 5.21, bo: 12.73, m: 10.36, I: 13.63692, Time: 0.06
It: 1080, Loss: 5.847e+04, bv: 5.21, bo: 12.67, m: 10.35, I: 13.63980, Time: 0.07
It: 1090, Loss: 5.844e+04, bv: 5.22, bo: 12.62, m: 10.35, I: 13.64266, Time: 0.06
It: 1100, Loss: 5.841e+04, bv: 5.22, bo: 12.57, m: 10.35, I: 13.64551, Time: 0.07
It: 1110, Loss: 5.841e+04, bv: 5.23, bo: 12.52, m: 10.35, I: 13.64835, Time: 0.07
It: 1120, Loss: 6.135e+04, bv: 5.23, bo: 12.46, m: 10.34, I: 13.65122, Time: 0.07
It: 1130, Loss: 5.896e+04, bv: 5.23, bo: 12.41, m: 10.34, I: 13.65450, Time: 0.07
It: 1140, Loss: 

It: 2040, Loss: 5.681e+04, bv: 5.13, bo: 10.22, m: 10.29, I: 13.91767, Time: 0.07
It: 2050, Loss: 5.682e+04, bv: 5.13, bo: 10.22, m: 10.29, I: 13.92049, Time: 0.07
It: 2060, Loss: 5.679e+04, bv: 5.13, bo: 10.21, m: 10.29, I: 13.92325, Time: 0.07
It: 2070, Loss: 5.673e+04, bv: 5.12, bo: 10.21, m: 10.29, I: 13.92597, Time: 0.07
It: 2080, Loss: 5.673e+04, bv: 5.12, bo: 10.20, m: 10.29, I: 13.92864, Time: 0.06
It: 2090, Loss: 5.674e+04, bv: 5.12, bo: 10.20, m: 10.29, I: 13.93128, Time: 0.07
It: 2100, Loss: 5.705e+04, bv: 5.12, bo: 10.20, m: 10.29, I: 13.93393, Time: 0.08
It: 2110, Loss: 5.706e+04, bv: 5.12, bo: 10.19, m: 10.29, I: 13.93676, Time: 0.07
It: 2120, Loss: 5.688e+04, bv: 5.12, bo: 10.19, m: 10.29, I: 13.93959, Time: 0.07
It: 2130, Loss: 5.665e+04, bv: 5.12, bo: 10.18, m: 10.29, I: 13.94237, Time: 0.07
It: 2140, Loss: 5.669e+04, bv: 5.12, bo: 10.18, m: 10.29, I: 13.94509, Time: 0.06
It: 2150, Loss: 5.665e+04, bv: 5.12, bo: 10.18, m: 10.29, I: 13.94777, Time: 0.07
It: 2160, Loss: 

It: 3060, Loss: 5.641e+04, bv: 5.28, bo: 10.21, m: 10.27, I: 14.16496, Time: 0.07
It: 3070, Loss: 5.681e+04, bv: 5.28, bo: 10.21, m: 10.27, I: 14.16691, Time: 0.06
It: 3080, Loss: 5.619e+04, bv: 5.28, bo: 10.22, m: 10.28, I: 14.16901, Time: 0.06
It: 3090, Loss: 5.625e+04, bv: 5.28, bo: 10.22, m: 10.28, I: 14.17100, Time: 0.09
It: 3100, Loss: 5.616e+04, bv: 5.29, bo: 10.22, m: 10.27, I: 14.17291, Time: 0.07
It: 3110, Loss: 5.620e+04, bv: 5.29, bo: 10.23, m: 10.27, I: 14.17470, Time: 0.07
It: 3120, Loss: 5.643e+04, bv: 5.29, bo: 10.23, m: 10.27, I: 14.17643, Time: 0.07
It: 3130, Loss: 5.667e+04, bv: 5.29, bo: 10.23, m: 10.27, I: 14.17835, Time: 0.06
It: 3140, Loss: 5.621e+04, bv: 5.29, bo: 10.23, m: 10.27, I: 14.18033, Time: 0.07
It: 3150, Loss: 5.621e+04, bv: 5.29, bo: 10.24, m: 10.27, I: 14.18218, Time: 0.07
It: 3160, Loss: 5.614e+04, bv: 5.29, bo: 10.24, m: 10.27, I: 14.18394, Time: 0.07
It: 3170, Loss: 5.617e+04, bv: 5.29, bo: 10.25, m: 10.27, I: 14.18559, Time: 0.06
It: 3180, Loss: 

It: 4080, Loss: 5.701e+04, bv: 5.33, bo: 10.54, m: 10.27, I: 14.31453, Time: 0.07
It: 4090, Loss: 5.629e+04, bv: 5.33, bo: 10.54, m: 10.28, I: 14.31644, Time: 0.06
It: 4100, Loss: 5.599e+04, bv: 5.33, bo: 10.54, m: 10.28, I: 14.31823, Time: 0.07
It: 4110, Loss: 5.587e+04, bv: 5.33, bo: 10.55, m: 10.27, I: 14.31973, Time: 0.08
It: 4120, Loss: 5.588e+04, bv: 5.33, bo: 10.55, m: 10.27, I: 14.32090, Time: 0.07
It: 4130, Loss: 5.587e+04, bv: 5.33, bo: 10.56, m: 10.27, I: 14.32178, Time: 0.07
It: 4140, Loss: 5.587e+04, bv: 5.33, bo: 10.56, m: 10.27, I: 14.32250, Time: 0.07
It: 4150, Loss: 5.590e+04, bv: 5.33, bo: 10.56, m: 10.27, I: 14.32319, Time: 0.06
It: 4160, Loss: 5.622e+04, bv: 5.34, bo: 10.56, m: 10.27, I: 14.32398, Time: 0.08
It: 4170, Loss: 5.625e+04, bv: 5.34, bo: 10.57, m: 10.27, I: 14.32539, Time: 0.07
It: 4180, Loss: 5.604e+04, bv: 5.34, bo: 10.57, m: 10.27, I: 14.32695, Time: 0.06
It: 4190, Loss: 5.585e+04, bv: 5.34, bo: 10.57, m: 10.27, I: 14.32834, Time: 0.07
It: 4200, Loss: 

...

It: 196320, Loss: 5.223e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.12995, Time: 0.07
It: 196330, Loss: 5.234e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.11843, Time: 0.07
It: 196340, Loss: 5.250e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13106, Time: 0.07
It: 196350, Loss: 5.278e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14409, Time: 0.07
It: 196360, Loss: 5.241e+04, bv: 4.92, bo: 11.06, m: 10.22, I: 11.14988, Time: 0.06
It: 196370, Loss: 5.237e+04, bv: 4.92, bo: 11.06, m: 10.22, I: 11.15094, Time: 0.08
It: 196380, Loss: 5.225e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14669, Time: 0.07
It: 196390, Loss: 5.243e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13654, Time: 0.06
It: 196400, Loss: 5.235e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14268, Time: 0.08
It: 196410, Loss: 5.223e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14012, Time: 0.07
It: 196420, Loss: 5.227e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.12797, Time: 0.07
It: 196430, Loss: 5.287e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.12879, Tim

It: 197310, Loss: 5.224e+04, bv: 4.93, bo: 11.06, m: 10.21, I: 11.14136, Time: 0.07
It: 197320, Loss: 5.228e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13169, Time: 0.07
It: 197330, Loss: 5.285e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13119, Time: 0.06
It: 197340, Loss: 5.239e+04, bv: 4.93, bo: 11.06, m: 10.21, I: 11.14065, Time: 0.07
It: 197350, Loss: 5.245e+04, bv: 4.92, bo: 11.06, m: 10.22, I: 11.14359, Time: 0.07
It: 197360, Loss: 5.236e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14392, Time: 0.08
It: 197370, Loss: 5.249e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14177, Time: 0.07
It: 197380, Loss: 5.231e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14327, Time: 0.07
It: 197390, Loss: 5.249e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13945, Time: 0.07
It: 197400, Loss: 5.232e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14483, Time: 0.07
It: 197410, Loss: 5.227e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14039, Time: 0.07
It: 197420, Loss: 5.267e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13424, Tim

It: 198300, Loss: 5.258e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13986, Time: 0.08
It: 198310, Loss: 5.224e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14765, Time: 0.07
It: 198320, Loss: 5.256e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14286, Time: 0.07
It: 198330, Loss: 5.229e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14711, Time: 0.08
It: 198340, Loss: 5.233e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14076, Time: 0.06
It: 198350, Loss: 5.225e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.12822, Time: 0.07
It: 198360, Loss: 5.303e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.12445, Time: 0.08
It: 198370, Loss: 5.287e+04, bv: 4.93, bo: 11.06, m: 10.21, I: 11.14596, Time: 0.07
It: 198380, Loss: 5.306e+04, bv: 4.92, bo: 11.06, m: 10.22, I: 11.16195, Time: 0.07
It: 198390, Loss: 5.339e+04, bv: 4.92, bo: 11.05, m: 10.22, I: 11.18654, Time: 0.07
It: 198400, Loss: 5.227e+04, bv: 4.92, bo: 11.05, m: 10.22, I: 11.21055, Time: 0.07
It: 198410, Loss: 5.249e+04, bv: 4.92, bo: 11.05, m: 10.22, I: 11.20895, Tim

It: 199290, Loss: 5.226e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14751, Time: 0.07
It: 199300, Loss: 5.238e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14119, Time: 0.07
It: 199310, Loss: 5.269e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13793, Time: 0.07
It: 199320, Loss: 5.234e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14702, Time: 0.07
It: 199330, Loss: 5.243e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14413, Time: 0.07
It: 199340, Loss: 5.248e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14590, Time: 0.07
It: 199350, Loss: 5.233e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14740, Time: 0.08
It: 199360, Loss: 5.248e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14348, Time: 0.06
It: 199370, Loss: 5.239e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14666, Time: 0.07
It: 199380, Loss: 5.236e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14475, Time: 0.08
It: 199390, Loss: 5.249e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14361, Time: 0.07
It: 199400, Loss: 5.237e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.14545, Tim

Loss: 5.222e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13892
Loss: 5.222e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13892
Loss: 5.222e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13892
Loss: 5.222e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13892
Loss: 5.222e+04, bv: 4.93, bo: 11.06, m: 10.22, I: 11.13892
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 52219.246094
  Number of iterations: 23
  Number of functions evaluations: 44


In [39]:
print('bv ',model.sess.run(model.bv)[0])
print('bo ',model.sess.run(model.bo)[0])
print('m ',model.sess.run(model.m)[0])
print('I ',model.sess.run(model.I)[0])

bv  4.929752
bo  11.0624695
m  10.217946
I  11.138924


In [ ]:
N_train = 5000
    
layers = [1, 10, 10, 6]

# Load Data
data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')
           
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2
    
N = X_star.shape[0]
T = t_star.shape[0]
    
# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T
    
UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T
    
x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1
    
u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1
    
    ######################################################################
    ######################## Noiseles Data ###############################
    ######################################################################
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

# Training
model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
model.train(200000)
    
# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]
    
u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]
    
# Prediction
u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star)
lambda_1_value = model.sess.run(model.lambda_1)
lambda_2_value = model.sess.run(model.lambda_2)
    
# Error
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    
print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error p: %e' % (error_p))    
print('Error l1: %.5f%%' % (error_lambda_1))                             
print('Error l2: %.5f%%' % (error_lambda_2))                  